In [57]:
using Compat.Test
using QuCircuit
import Base:sparse
import QuCircuit: GateType
rot_basis() = gate(GateType{:RotBasis}) |> roll

rot_basis (generic function with 1 method)

In [58]:
import QuCircuit: PrimitiveBlock
# use different types for parameters and matrix
#= ideal interface should be
@sparsegate begin
    RotBasis(theta::T, phi::T) where T = sparse(chain(rot(T, :X, -theta), rot(T, :Z, -phi)))
end
=#
struct RotBasis{Tp, Tm} <: PrimitiveBlock{1, Tm}
    theta::Tp
    phi::Tp
end

In [59]:
# chain -> *
sparse(rb::RotBasis{T}) where T = sparse(chain(rot(T, :X, -rb.theta), rot(T, :Z, -rb.phi)))

sparse (generic function with 36 methods)

In [60]:
rot(Float64, :X, 0.3)

Rx{Float64}: 0.3

In [61]:
reg = zero_state(3)
rb = rot_basis()(4)
rb(reg)

LoadError: [91mMethodError: no method matching nqubits(::Type{QuCircuit.GateType{:RotBasis}})[0m
Closest candidates are:
  nqubits([91m::Type{QuCircuit.GateType{:X}}[39m) at /home/leo/jcode/QuCircuit.jl/src/Blocks/ConstantGate.jl:19
  nqubits([91m::Type{QuCircuit.GateType{:Y}}[39m) at /home/leo/jcode/QuCircuit.jl/src/Blocks/ConstantGate.jl:19
  nqubits([91m::Type{QuCircuit.GateType{:Z}}[39m) at /home/leo/jcode/QuCircuit.jl/src/Blocks/ConstantGate.jl:19
  ...[39m

In [62]:
function get_axis(A::AbstractArray{T, N}, dim, i) where {T, N}
	getindex(A, ntuple(d->d==dim?i:Colon(), Val{N})...)
end

get_axis (generic function with 1 method)

In [6]:
# translation between polar angle and len-2 complex vector.
function u2polar(vec)
    ratio = get_axis(vec, 1, 2)./get_axis(vec, 1, 1)
    @. [atan(abs(ratio))'*2; angle(ratio)']
end

function polar2u(polar)
    theta, phi = get_axis(polar, 1, 1)', get_axis(polar, 1, 2)'
    @. [cos(theta/2)*exp(-im*phi/2); sin(theta/2)*exp(im*phi/2)]
end

# random polar basis, n-> number of basis
randpolar(params...) = rand(2, params...)*pi

randpolar (generic function with 1 method)

In [7]:
polar = randpolar(10)
print(size(polar))
@test all(isapprox.(polar |> polar2u |> u2polar, polar))

(2, 10)

Test Passed


In [8]:
rb = rot_basis()
angles = randpolar()
# prepair a state in the angles direction.
psi = angles |> polar2u |> Register
# rotate to the same direction for measurements.
dispatch!(rb, vec(angles))

@test state(psi |> rb) ≈ [1,0]

LoadError: [91mUndefVarError: basic_gate not defined[39m

In [55]:
# Grover search block
# psi and oracle are needed
struct Reflect{N, T} <: PrimitiveBlock{N, T}
    state :: Vector{T}
end

Reflect(state::Vector{T}) where T = Reflect{log2i(length(state)), T}(state)

# NOTE: this should not be matrix multiplication based
function apply!(r::Register, g::Reflect)
    println(size(g.state' * r.state), reshape(g.state, :, 1)|>size)
    @views r.state[:,:] .= 2* (g.state'*r.state) .* reshape(g.state, :, 1) - r.state
    
end

function show(io::IO, g::Reflect{N, T}) where {N, T}
    print("Reflect(N = $N")
end

show (generic function with 1 method)

In [56]:
rf = Reflect(randn(1<<3)*im)
apply!(reg, rf)

(1, 1)(8, 1)


8×1 SubArray{Complex{Float64},2,Array{Complex{Float64},2},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}}},true}:
 -45.0013+0.0im
  64.4213+0.0im
  29.4315+0.0im
 -25.3453+0.0im
 -50.3702+0.0im
  31.9865+0.0im
  -102.02+0.0im
 -78.1152+0.0im

In [28]:
inference_oracle(locs) = X(i) |> C(locs)

inference_oracle (generic function with 1 method)

In [118]:
num_bit = 9
b = basis(num_bit)
psi0 = rand_state(num_bit)
psi = copy(psi0)
psi |> inference_oracle([9, 2, -3])
psi0[indices_with(num_bit, [9, 2, 3], [1, 1, 0])] *= -1
@test psi == psi0

LoadError: [91mUndefVarError: basis not defined[39m